In [1]:
import math
import time
import random
import copy

In [2]:
pessoas = [('Amanda', 'CWB'),
           ('Pedro', 'GIG'),
           ('Marcos', 'POA'),
           ('Priscila', 'FLN'),
           ('Jessica', 'CNF'),
           ('Paulo', 'GYN')]

destino = 'GRU'

In [3]:
voos = {}

for l in open('voos.txt'):
    origem, dest, saida, chegada, preco = l.split(',')
    voos.setdefault((origem, dest), [])
    voos[(origem, dest)].append((saida, chegada, int(preco)))

In [4]:
def print_agenda(agenda):
    for idx, voo in enumerate(agenda):
        nome = pessoas[idx][0]
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        print("%10s%10s %5s-%5s R$%3s %5s-%5s R$%3s" % (nome, origem, ida[0], ida[1], ida[2],
                                                        volta[0], volta[1], volta[2]))

agenda = [[1,3], [3, 2], [7,3], [6,3], [2,4], [5,3]]

print_agenda(agenda)

    Amanda       CWB  8:04-10:11 R$ 95 10:33-12:03 R$ 74
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 17:08-19:08 R$262 10:32-13:16 R$139
  Priscila       FLN 15:34-18:11 R$326 11:08-14:38 R$262
   Jessica       CNF  9:42-11:32 R$169 12:08-14:47 R$231
     Paulo       GYN 13:37-15:08 R$250 11:07-13:24 R$171


In [5]:
def get_minutos(hora:str):
    tempo = time.strptime(hora, '%H:%M')
    return tempo[3] * 60 + tempo[4]

get_minutos('1:10')

70

In [6]:
def custo(solucao:list):
    preco_total = 0
    # menor possível
    ultima_chegada = 0
    # maior possível
    primeira_partida = 1439 #23:59 em minutos
    total_espera = 0

    for idx, voo in enumerate(solucao):
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        preco_total += ida[2]
        preco_total += volta[2]

        # atualiza a ultima chegada ao aeroporto
        if ultima_chegada < get_minutos(ida[1]):
            ultima_chegada = get_minutos(ida[1])

        # atualiza com a primeira chegada ao aeroporto
        if primeira_partida > get_minutos(volta[0]):
            primeira_partida = get_minutos(volta[0])
        
    for idx, voo in enumerate(solucao):
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        # calcula o tanto de espera no aeroporto
        total_espera += ultima_chegada - get_minutos(ida[1])
        total_espera += get_minutos(volta[0]) - primeira_partida

    # se virou um dia, adiciona penalidade em dinheiro
    if ultima_chegada > primeira_partida:
        preco_total += 50

    return preco_total + total_espera

In [7]:
custo(agenda)

4472

## Busca randômica

In [8]:
def busca_randomica(dominio, funcao_custo, numero_solucao):
    melhor_custo = 99999999
    melhor_solucao = None

    for i in range(numero_solucao):
        solucao = [[random.randint(dominio[i][0], dominio[i][1]), random.randint(dominio[i][0], dominio[i][1])] for i in range(len(dominio))]

        custo = funcao_custo(solucao)

        if custo < melhor_custo:
            melhor_custo = custo
            melhor_solucao = solucao

    print('Melhor solução: ', melhor_solucao)
    print('Custo solução: ', melhor_custo)
    print_agenda(melhor_solucao)

    return melhor_solucao

# para cada pessoa, tem-se o numero possivel de voos.
dominio = [[0, 9]] * (len(pessoas))

busca_randomica(dominio, funcao_custo=custo, numero_solucao=1000)

Melhor solução:  [[7, 5], [6, 2], [5, 3], [0, 1], [5, 1], [7, 3]]
Custo solução:  4065
    Amanda       CWB 17:11-18:30 R$108 13:39-15:30 R$ 74
     Pedro       GIG 15:44-18:55 R$382  9:49-13:51 R$229
    Marcos       POA 13:40-15:38 R$137 10:32-13:16 R$139
  Priscila       FLN  6:25- 9:30 R$335  8:23-11:07 R$143
   Jessica       CNF 14:22-16:32 R$126  7:50-10:08 R$164
     Paulo       GYN 16:51-19:09 R$147 11:07-13:24 R$171


[[7, 5], [6, 2], [5, 3], [0, 1], [5, 1], [7, 3]]

## Hill Climb

In [9]:
def hill_climb(domino, funcao_custo):
    # gera uma solucao randomica para iniciar o algoritmo
    solucao = busca_randomica(dominio=dominio, funcao_custo=funcao_custo,  numero_solucao=1000)

    print('Solucao aleatoria: ', solucao)

    while True:
        vizinhos = []
    
        for i in range(len(dominio)):
            # le cada valor da tupla
            for j in range(len(dominio[i])):
                 # verifica se o valor é menor que o valor maximo do dominio
                if solucao[i][j] < dominio[i][1]:
                    # faz uma copida da lista a ser modificado
                    novo_vizinho = copy.deepcopy(solucao)
                    # modifica o valor da solucao na posica [i][j]
                    novo_vizinho[i][j] = novo_vizinho[i][j] + 1
                    # adiciona a lista modificada aos vizinhos
                    vizinhos.append(novo_vizinho)
            
                # verifica se o valor é maior que o valor minimo do dominio
                if solucao[i][j] > dominio[i][0]:
                    # faz uma copida da lista a ser modificado
                    novo_vizinho = copy.deepcopy(solucao)
                    # modifica o valor da solucao na posica [i][j]
                    novo_vizinho[i][j] = novo_vizinho[i][j] - 1
                    # adiciona a lista modificada aos vizinhos
                    vizinhos.append(novo_vizinho)
        # verifica o custo da solucao aleatorea
        atual = funcao_custo(solucao)
        # adiciona o custo da funcao aleatórea a melhor
        melhor = atual

        for i in range(len(vizinhos)):
            # print(vizinhos[i])
            custo = funcao_custo(vizinhos[i])

            if custo < melhor:
                # atualiza para o melhor custo
                melhor = custo
                # registra a melhor solucao
                solucao = vizinhos[i]

        # melhro solucao encontrada
        if melhor == atual:
            break

    print('Melhor solucão: ', solucao)
    print('Melhor custo: ', melhor)
    print_agenda(agenda=solucao)
    
    return solucao

In [10]:
hill_climb(domino=dominio, funcao_custo=custo)

Melhor solução:  [[9, 1], [8, 2], [4, 2], [7, 1], [5, 1], [9, 2]]
Custo solução:  4001
    Amanda       CWB 20:17-22:22 R$102  8:23-10:28 R$149
     Pedro       GIG 18:26-21:29 R$464  9:49-13:51 R$229
    Marcos       POA 12:08-14:59 R$149  9:58-12:56 R$249
  Priscila       FLN 17:07-20:04 R$291  8:23-11:07 R$143
   Jessica       CNF 14:22-16:32 R$126  7:50-10:08 R$164
     Paulo       GYN 20:05-22:06 R$261  9:31-11:43 R$210
Solucao aleatoria:  [[9, 1], [8, 2], [4, 2], [7, 1], [5, 1], [9, 2]]
Melhor solucão:  [[9, 1], [8, 2], [8, 1], [9, 1], [6, 1], [9, 1]]
Melhor custo:  3004
    Amanda       CWB 20:17-22:22 R$102  8:23-10:28 R$149
     Pedro       GIG 18:26-21:29 R$464  9:49-13:51 R$229
    Marcos       POA 18:35-20:28 R$204  8:19-11:16 R$122
  Priscila       FLN 19:53-22:21 R$173  8:23-11:07 R$143
   Jessica       CNF 15:58-18:40 R$173  7:50-10:08 R$164
     Paulo       GYN 20:05-22:06 R$261  8:04-10:59 R$136


[[9, 1], [8, 2], [8, 1], [9, 1], [6, 1], [9, 1]]

## Simulated Annealing

In [64]:
def simulated_annealing(dominio, funcao_custo, temperatura = 10000.0, taxa_resfriamento = 0.95, passo = 1):
    # pega uma solucao randomica para inicializar o algoritmo
    solucao = busca_randomica(dominio=dominio, funcao_custo=funcao_custo,  numero_solucao=1000)

    while temperatura > 0.1:
        # escolhe uma posicao aleatoria para fazer a modificacao
        random_idx = random.randint(0, len(dominio) - 1)
        random_idx_dupla = random.randint(0, 1)

        # escolhe a direcao a que quer mudar. Caso passo = 1, posicao randomica entre -1 e 1
        direcao = random.randint(-passo, passo) 

        solucao_mod = copy.deepcopy(solucao)
        # modifica a posicao aleatoria gerada
        solucao_mod[random_idx][random_idx_dupla] += direcao

        # se for negativo, coloca 0
        if solucao_mod[random_idx][random_idx_dupla] < dominio[random_idx][0]:
            solucao_mod[random_idx][random_idx_dupla] = dominio[random_idx][0]
        # se for maior que o max do dominio, coloca o max
        elif solucao_mod[random_idx][random_idx_dupla] > dominio[random_idx][1]:
            solucao_mod[random_idx][random_idx_dupla] = dominio[random_idx][1]

        # calcula os custos
        custo_solucao = funcao_custo(solucao)
        custo_solucao_mod = funcao_custo(solucao_mod)
        
        # calcula a probabilidade de escolher uma solucao 'pior'
        prob = pow(math.e, (-custo_solucao_mod - custo_solucao) / temperatura)

        if custo_solucao_mod < custo_solucao or random.random() < prob:
            solucao = solucao_mod

        # diminui a temperatura
        temperatura *= taxa_resfriamento

    print('Melhor solucão: ', solucao)
    print('Melhor custo: ', custo_solucao)
    print_agenda(agenda=solucao)

    return solucao

In [73]:
simulated_annealing(dominio=dominio, funcao_custo=custo)

Melhor solução:  [[1, 6], [1, 3], [2, 7], [2, 6], [1, 7], [3, 3]]
Custo solução:  4158
    Amanda       CWB  8:04-10:11 R$ 95 15:25-16:58 R$ 62
     Pedro       GIG  7:53-11:37 R$433 10:51-14:16 R$256
    Marcos       POA  9:15-12:14 R$247 16:33-18:15 R$253
  Priscila       FLN  9:15-12:29 R$225 15:23-18:49 R$150
   Jessica       CNF  8:25-10:34 R$157 17:06-20:00 R$ 95
     Paulo       GYN 11:08-13:07 R$175 11:07-13:24 R$171
Melhor solucão:  [[2, 3], [0, 3], [1, 5], [2, 6], [2, 3], [2, 3]]
Melhor custo:  2888
    Amanda       CWB  9:45-11:50 R$172 10:33-12:03 R$ 74
     Pedro       GIG  6:12-10:22 R$230 10:51-14:16 R$256
    Marcos       POA  8:27-10:45 R$139 13:37-15:33 R$142
  Priscila       FLN  9:15-12:29 R$225 15:23-18:49 R$150
   Jessica       CNF  9:42-11:32 R$169 10:33-13:11 R$132
     Paulo       GYN  9:15-12:03 R$ 99 11:07-13:24 R$171


[[2, 3], [0, 3], [1, 5], [2, 6], [2, 3], [2, 3]]